In [2]:
import xarray as xr
import torch
from torch.utils.data import DataLoader
from xbatcher import BatchGenerator

from utils.general import load_config

config = load_config()

In [3]:
hr_data = xr.open_dataset(
    "/home/ubuntu/project/destine-super-resolution/ScenarioMIP-SSP3-7.0-IFS-NEMO-0001-high-sfc-v0.zarr",
    engine="zarr",
    chunks={})

lr_data = xr.open_dataset(
    "/home/ubuntu/project/destine-super-resolution/ScenarioMIP-SSP3-7.0-IFS-NEMO-0001-standard-sfc-v0.zarr",
    engine="zarr",
    chunks={})
lr_data

<xarray.Dataset> Size: 9GB
Dimensions:    (time: 240, latitude: 512, longitude: 1025)
Coordinates:
  * latitude   (latitude) float64 4kB -90.0 -89.65 -89.3 ... 89.3 89.65 90.0
  * longitude  (longitude) float64 8kB -180.0 -179.6 -179.3 ... 179.6 180.0
    step       timedelta64[ns] 8B ...
    surface    float64 8B ...
  * time       (time) datetime64[ns] 2kB 2020-01-01 ... 2020-01-10T23:00:00
Data variables: (12/18)
    blh        (time, latitude, longitude) float32 504MB dask.array<chunksize=(240, 128, 128), meta=np.ndarray>
    chnk       (time, latitude, longitude) float32 504MB dask.array<chunksize=(240, 128, 128), meta=np.ndarray>
    d2m        (time, latitude, longitude) float32 504MB dask.array<chunksize=(240, 128, 128), meta=np.ndarray>
    hcc        (time, latitude, longitude) float32 504MB dask.array<chunksize=(240, 128, 128), meta=np.ndarray>
    lcc        (time, latitude, longitude) float32 504MB dask.array<chunksize=(240, 128, 128), meta=np.ndarray>
    mcc        (time, latitude, longitude) float32 504MB dask.array<chunksize=(240, 128, 128), meta=np.ndarray>
    ...         ...
    tciw       (time, latitude, longitude) float32 504MB dask.array<chunksize=(240, 128, 128), meta=np.ndarray>
    tclw       (time, latitude, longitude) float32 504MB dask.array<chunksize=(240, 128, 128), meta=np.ndarray>
    tcwv       (time, latitude, longitude) float32 504MB dask.array<chunksize=(240, 128, 128), meta=np.ndarray>
    tprate     (time, latitude, longitude) float32 504MB dask.array<chunksize=(240, 128, 128), meta=np.ndarray>
    u10        (time, latitude, longitude) float32 504MB dask.array<chunksize=(240, 128, 128), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 504MB dask.array<chunksize=(240, 128, 128), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_edition:            2
    GRIB_subCentre:          1003
    history:                 2024-03-27T16:00 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts

In [4]:
hr_data = hr_data[config['dataset']['data_target']]
lr_data = lr_data[config['dataset']['data_variable']]

In [5]:
import xarray as xr
from xbatcher import BatchGenerator

In [6]:
batch_generator_lr = BatchGenerator(lr_data, input_dims={"time": config['training']['batch_size'], "latitude":  lr_data.sizes['latitude'], "longitude": lr_data.sizes['longitude']})
batch_generator_hr = BatchGenerator(hr_data, input_dims={"time": config['training']['batch_size'], "latitude":  hr_data.sizes['latitude'], "longitude": hr_data.sizes['longitude']})

In [7]:
GPU_DEVICE = 0
device = torch.device("cuda",GPU_DEVICE)

import matplotlib.pyplot as plt

In [9]:
%%timeit
# Iterate through one batch
for batch in batch_generator_lr:
    data  = batch.load()
    data = data.to_array().values
    data = torch.tensor(data)
    data.to(device)
    break

212 ms ± 7.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# cloud based 20.3 s ± 437 ms per loop (mean ± std. dev. of 7 runs, 1 loop each) (24 x 4096 x 8193)
# 3.23 s ± 41.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each) (24 x 4096 x 8193)